In [1]:
from pandas_datareader import data,wb
import pandas as pd
import numpy as np
import plotly
import cufflinks as cf
from apscheduler.schedulers.background import BackgroundScheduler
cf.go_offline()
import datetime
%matplotlib inline
start = datetime.datetime(2022,9,16)
end = datetime.datetime(2022,9,16)
share = data.DataReader('^NSEI', 'yahoo', start, end)

In [2]:
nifty = pd.read_excel('INDEX.xlsx','NIFTY50')
nifty.set_index('Ticker',inplace=True)
nifty.head()

,Weightage
Ticker,
ADANIPORTS.NS,0.73
AXISBANK.NS,2.54
ASIANPAINT.NS,2.00
BAJAJFINSV.NS,1.21
BAJAJ-AUTO.NS,0.68


In [3]:
tracking_stocks_2=pd.DataFrame()
tracking_stocks_1=pd.DataFrame()
tracking_stocks_1.index=nifty.index
df = pd.DataFrame(columns=['Sigmaval'], index=pd.to_datetime([]))
df.index.names = ['Current Time']
tracking_stocks_1['Price']=0
tracking_stocks_1['Volume']=0
convert_dict = {'Price': float,
                'Volume': float
               }
tracking_stocks_1=tracking_stocks_1.astype(convert_dict)
current=0
past=0
diff=0

In [4]:
sched = BackgroundScheduler()
@sched.scheduled_job('interval', seconds=300)
def timed_job():
    global current
    global past
    global nifty
    global diff
    global start
    global end
    global tracking_stocks_2
    global tracking_stocks_1
    global df
    for stock in list(nifty.index):
        nifty_share=pd.DataFrame()
        nifty_share = data.DataReader(stock, 'yahoo', start, end)
        tracking_stocks_1.loc[stock]['Price']=(nifty_share.loc[end]['Adj Close'])
        tracking_stocks_1.loc[stock]['Volume']=(nifty_share.loc[end]['Volume'])
    tracking_stocks_1=tracking_stocks_1.merge(nifty, how='inner',on='Ticker')
    tracking_stocks_1['Score']=((tracking_stocks_1['Price']*tracking_stocks_1['Volume']*tracking_stocks_1['Weightage'])/10000000)
    if len(list(tracking_stocks_2.columns))==0:
        tracking_stocks_2=tracking_stocks_1.copy()
        current=tracking_stocks_1['Score'].sum()
        tracking_stocks_1=pd.DataFrame()
        tracking_stocks_1.index=nifty.index
        tracking_stocks_1['Price']=0
        tracking_stocks_1['Volume']=0
        convert_dict = {'Price': float,'Volume': float}
        tracking_stocks_1=tracking_stocks_1.astype(convert_dict)
        df.loc[pd.Timestamp('now')] = pd.Series([current], ['Sigmaval']) 
    else:
        current=tracking_stocks_1['Score'].sum()
        past=tracking_stocks_2['Score'].sum()
        diff=current-past
        tracking_stocks_2=tracking_stocks_1.copy()
        tracking_stocks_1=pd.DataFrame()
        tracking_stocks_1.index=nifty.index
        tracking_stocks_1['Price']=0
        tracking_stocks_1['Volume']=0
        convert_dict = {'Price': float,'Volume': float}
        tracking_stocks_1=tracking_stocks_1.astype(convert_dict)
        df.loc[pd.Timestamp('now')] = pd.Series([diff], ['Sigmaval'])
sched.start()    

In [5]:
df.reset_index().loc[1:].set_index("Current Time").iplot(kind='line')

In [85]:
df

,Sigmaval
Current Time,
2022-09-16 12:32:21.918194,39643.006421
2022-09-16 12:37:37.557856,523.319825
2022-09-16 12:42:19.876605,514.942950
2022-09-16 12:47:26.054565,471.717901
2022-09-16 12:52:25.847946,598.584157
2022-09-16 12:57:20.173786,569.608456
2022-09-16 13:02:23.504690,628.669110
2022-09-16 13:07:28.642011,735.060604
2022-09-16 13:12:22.781792,798.160427
